<a href="https://colab.research.google.com/github/taniae27/AprendizajeProfundo/blob/main/Tarea_1_Retropropagaci%C3%B3n_en_red_densa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2. Retropropagación en red densa

Programa el algoritmo de retropropagación usando NumPy para una tarea de clasificación binaria presuponiendo una red densa con dos capas ocultas. Esta red tiene una función de activación logística en todas sus neuronas y se entrena minimizando la función de pérdida de entropía cruzada
binaria. Describe las fórmulas y reglas de actualización de los pesos y sesgos de cada capa y entrena
y evalúa la red en algún conjunto de datos.

In [1]:
import numpy as np

Nuestra red neuronal densa está compuesta por una capa de 2 entradas ($x_1$ y $x_2$), una capa oculta con 10 neuronas con función de activación sigmoide y una capa de salida con una sola neurona con función de activación sigmoide. Esta función de activación se define como:

$$
\sigma(z) = \frac{1}{1 + e^{-z}}
$$

In [2]:
def sigmoide(z):
    return 1 / (1 + np.exp(-z))

La función sigmoide tiene una derivada que está expresada en términos de la misma función, esto es, 

$$
\frac{\partial \sigma (z)}{\partial z} = \sigma(z) (1 - \sigma(z))
$$

In [3]:
def derivada_sigmoide(x):
    return np.multiply(sigmoide(x), (1.0 - sigmoide(x)))

Usaremos la función de pérdida de entropía cruzada binaria:

$$
ECB(\mathbf{y}, \mathbf{\hat{y}})  = -\sum_{i=1}^N \left[ y^{(i)} \log \hat{y}^{(i)} + (1 - y^{(i)}) \log (1 - \hat{y}^{(i)}) \right]
$$

In [4]:
def ECB(y, p):
    p[p == 0] = np.nextafter(0., 1.)
    p[p == 1] = np.nextafter(1., 0.)
    return -(np.log(p[y == 1]).sum() + np.log(1 - p[y == 0]).sum())

Posteriormente calcularemos la exactitud para medir el rendimiento del modelo aprendido por la red neuronal densa:

$$
exactitud = \frac{correctos}{total}
$$

In [5]:
def exactitud(y, y_predicha):
    return (y == y_predicha).mean() * 100

Ahora, definimos la función que propaga hacia adelante una entrada $\mathbf{x}^{i}$. Como la red está compuesta de 3 capas densas (2 ocultas y 1 de salida), tenemos 3 matrices de pesos con sus correspondientes vectores de sesgos $\{\mathbf{W}^{\{1\}}, \mathbf{b}^{\{1\}}\}$ , $\{\mathbf{W}^{\{2\}}, \mathbf{b}^{\{2\}}\}$ y $\{\mathbf{W}^{\{3\}}, \mathbf{b}^{\{3\}}\}$ de las capas ocultas y la capa de salida respectivamente. Así, podemos llevar a cabo la propagación hacia adelante en esta red de la siguiente manera:

$$
	\begin{split}
				\mathbf{a}^{\{1\}} & =  \mathbf{x}^{(i)} \\
				\mathbf{z}^{\{2\}} & =  \mathbf{W}^{\{1\}} \cdot \mathbf{a}^{\{1\}} + \mathbf{b}^{\{1\}}\\
				\mathbf{a}^{\{2\}} & =  \sigma(\mathbf{z}^{\{2\}}) \\
				\mathbf{z}^{\{3\}} & =  \mathbf{W}^{\{2\}} \cdot \mathbf{a}^{\{2\}}  + \mathbf{b}^{\{2\}}\\
				\mathbf{a}^{\{3\}} & =  \sigma(\mathbf{z}^{\{3\}})\\
                \mathbf{z}^{\{4\}} & =  \mathbf{W}^{\{3\}} \cdot \mathbf{a}^{\{3\}}  + \mathbf{b}^{\{3\}}\\
				\mathbf{a}^{\{4\}} & =  \sigma(\mathbf{z}^{\{4\}})\\
				\hat{y}^{(i)} & =  \mathbf{a}^{\{4\}}
			\end{split}
      $$

In [6]:
def hacia_adelante(x, W1, b1, W2, b2, W3, b3):
    z2 = np.dot(W1.T, x[:, np.newaxis]) + b1
    a2 = sigmoide(z2)
    z3 = np.dot(W2.T, a2) + b2
    a3 = sigmoide(z3)
    z4 = np.dot(W3.T, a3) + b3
    y_hat = sigmoide(z4)
    return z2, a2, z3, a3, z4, y_hat

FinalmSe define la función para entrenar nuestra red neuronal usando gradiente descendente. Para calcular el gradiente de la función de pérdida respecto a los pesos y sesgos en cada capa empleamos el algoritmo de retropropagación.



In [7]:
def retropropagacion(X, y, alpha = 0.01, n_epocas = 100, n_ocultas = 3):
    n_ejemplos = X.shape[0]
    n_entradas = X.shape[1]
        
    # Inicialización de las matrices de pesos W y V
    W1 = np.sqrt(1.0 / n_entradas) * np.random.randn(n_entradas, n_ocultas)
    b1 = np.zeros((n_ocultas, 1))
    
    W2 = np.sqrt(1.0 / n_entradas) * np.random.randn(n_ocultas, n_ocultas)
    b2 = np.zeros((n_ocultas, 1))
    
    W3 = np.sqrt(1.0 / n_ocultas) * np.random.randn(n_ocultas, 1)
    b3 = np.zeros((1, 1))
    
    perdidas = np.zeros((n_epocas))
    exactitudes = np.zeros((n_epocas))
    y_predicha = np.zeros((y.shape))
    for i in range(n_epocas):
        for j in range(n_ejemplos):
            z2, a2, z3, a3, z4, y_hat = hacia_adelante(X[j], W1, b1, W2, b2, W3, b3)

            # cálculo de gradientes para W3 y b3 por retropropagación
            dz4 = y_hat - y[j]
            dW3 = np.outer(a3, dz4) * derivada_sigmoide(z4)
            db3 = dz4

            # cálculo de gradientes para W2 y b2 por retropropagación
            dz3 = np.dot(W3, dz4) * derivada_sigmoide(z3)
            dW2 = np.outer(a2, dz3)
            db2 = dz3
            
            # cálculo de gradientes para W1 y b1 por retropropagación
            dz2 = np.dot(W2, dz3) * derivada_sigmoide(z2)
            dW1 = np.outer(X[j], dz2)
            db1 = dz2

            # la actualización de los parámetros
            # debe hacerse de forma simultánea
            W3 = W3 - alpha * dW3
            b3 = b3 - alpha * db3              
            W2 = W2 - alpha * dW2
            b2 = b2 - alpha * db2
            W1 = W1 - alpha * dW1
            b1 = b1 - alpha * db1

            y_predicha[j] = y_hat
            
        # calcula la pérdida en la época
        perdidas[i] = ECB(y, y_predicha)
        exactitudes[i] = exactitud(y, np.round(y_predicha))
        print('Epoch {0}: Pérdida = {1} Exactitud = {2}'.format(i, 
                                                              perdidas[i], 
                                                              exactitudes[i]))

    return W1, W2, W3, perdidas, exactitudes

## Prueba considerando la operación XOR.

In [8]:
# ejemplo (XOR)
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0, 1, 1, 0]]).T

Finalmente, entrenamos nuestra red con estos ejemplos por 50 épocas usando una tasa de aprendizaje $\alpha = 2$.

In [10]:
np.random.seed(0)
W1, W2, W3, perdidas, exactitudes = retropropagacion(X, 
                                                 y, 
                                                 alpha = 2, 
                                                 n_epocas = 50,
                                                 n_ocultas = 5)

Epoch 0: Pérdida = 4.780265814405273 Exactitud = 25.0
Epoch 1: Pérdida = 4.443202515632271 Exactitud = 50.0
Epoch 2: Pérdida = 4.376765668042093 Exactitud = 50.0
Epoch 3: Pérdida = 4.324670862838479 Exactitud = 50.0
Epoch 4: Pérdida = 4.280614153080169 Exactitud = 50.0
Epoch 5: Pérdida = 4.24207793556266 Exactitud = 50.0
Epoch 6: Pérdida = 4.206816320556837 Exactitud = 50.0
Epoch 7: Pérdida = 4.173139764961292 Exactitud = 50.0
Epoch 8: Pérdida = 4.140565944389521 Exactitud = 50.0
Epoch 9: Pérdida = 4.110251098434749 Exactitud = 50.0
Epoch 10: Pérdida = 4.084226040009762 Exactitud = 50.0
Epoch 11: Pérdida = 4.063649308164782 Exactitud = 50.0
Epoch 12: Pérdida = 4.048109145320366 Exactitud = 50.0
Epoch 13: Pérdida = 4.036426215110716 Exactitud = 50.0
Epoch 14: Pérdida = 4.0274754240134945 Exactitud = 50.0
Epoch 15: Pérdida = 4.020436300558166 Exactitud = 50.0
Epoch 16: Pérdida = 4.014757276844345 Exactitud = 50.0
Epoch 17: Pérdida = 4.010071200970525 Exactitud = 50.0
Epoch 18: Pérdida = 